In [9]:
import pandas as pd
import numpy as np
import glob, warnings
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from datetime import datetime

warnings.filterwarnings(action='ignore')

from IPython.core.display import display, HTML

display(HTML("<style>.container { width: 80% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#legend.fontsize	범례의 글꼴 크기 
#legend.title_fontsize
COLOR='black'
parameters = {'axes.labelsize': 25,
          'axes.titlesize': 35,
          'xtick.labelsize' : 15,
          'ytick.labelsize' : 15,
          'text.color' : COLOR, 'axes.labelcolor' : COLOR, 'xtick.color': COLOR, 'ytick.color' : COLOR}


plt.rcParams.update(parameters)

import matplotlib.ticker as ticker
import matplotlib.dates as md
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

from itertools import chain
from sklearn.cluster import DBSCAN

In [10]:
#C:\Users\USER\Desktop\데이터\알찬\eggcounter\20220204
data_dir = 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\**/'
FileList = glob.glob(data_dir + '*N00.txt')
img_namess = FileList
print(img_namess)

['C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220101_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220102_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220103_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220104_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220105_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220106_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220107_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220108_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220109_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220110_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220111_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220125\\20220112_G00_N00.txt', 'C:\\Users\\USER\\Desktop\\

In [14]:
egg_df_dict= {}
egg_total_df = pd.DataFrame(columns=["날짜", "왕란", "특란", "대란", "중란", "소란", "경란", "파란","혈란","Sub Total", "Total", "CSV Total", "CSV Good Sum", "CSV Crack Sum"])
today = datetime.now()
today = today.strftime("%Y%m%d")

for idx, file_name in enumerate(img_namess):
    date = file_name.split("\\")[-1].split("_")[0]
    if date != today :
        continue
    
    egg_dict = {}
    total_count = 0
    with open(file_name, "r") as file :
        line_number= 1

        while True :
            line = file.readline()

            if not line :
                break

            line_strip = line.strip()

            if line_strip == '':
                continue

            else :
                if line_number == 1 :
                    value = line.strip().split(",")
                    for i in range(0, len(value)) :
                        if i == 0 :
                            continue

                        val = value[i].split(":")
                        total_count = val[1]
                    #pass
                else :
                    value = line.strip().split(":")
                    egg_dict[value[0]] = value[1]

            line_number +=1
    
    # DataFrame 생성
    egg_df = pd.DataFrame([egg_dict]).T
    egg_df = egg_df[egg_df.columns.values.tolist()[0]].str.split(',')
    egg_df = egg_df.apply(lambda x: pd.Series(x))
    egg_df = egg_df.rename(columns={0:"왕란", 1:"특란", 2:"대란", 3:"중란", 4:"소란", 5:"경란", 6:"파란", 7:"혈란"})
    egg_df.loc['단위중량'] = egg_df.apply(lambda x : round(int(x.loc['등급중량']) / int(x.loc['등급수량']) , 2) if (int(x.loc['등급중량'])!=0) & (int(x.loc['등급수량'])!=0)else 0 )
    egg_df = egg_df.astype(int)
    egg_df['Sub Total'] = egg_df['왕란'] + egg_df['특란'] + egg_df['대란']  + egg_df['중란'] + egg_df['소란'] + egg_df['경란']
    egg_df['Total'] = total_count
    egg_df = egg_df.loc['등급수량']
    
    # CSV 데이터 붙이기
    egg_df_csv = pd.read_csv(file_name.replace("_G00_N00.txt",".csv"), engine='python')
    egg_df_csv = egg_df_csv[['Total','Good','Crack']]

    egg_df['CSV Total'] = egg_df_csv['Total'].sum()
    egg_df['CSV Good Sum'] = egg_df_csv['Good'].sum()
    egg_df['CSV Crack Sum'] = egg_df_csv['Crack'].sum()
    egg_df['날짜'] = date

    egg_df = egg_df[["날짜", "왕란", "특란", "대란", "중란", "소란", "경란","파란","혈란", "Sub Total", "Total", "CSV Total", "CSV Good Sum", "CSV Crack Sum"]]

    egg_total_df = egg_total_df.append(egg_df)
    egg_total_df = egg_total_df.reset_index(drop = True)
    

egg_total_df
#egg_total_df[(egg_total_df['소란'] > 0) | (egg_total_df['경란'] > 0)]

,날짜,왕란,특란,대란,중란,소란,경란,파란,혈란,Sub Total,Total,CSV Total,CSV Good Sum,CSV Crack Sum
0,20220316,369,11418,2050,280,0,0,64,582,14117,14784,14412,13721,691


In [12]:
#df.to_csv("egg counter.csv", index=False, columns=['date', 'image_name','pixel_mean', 'pixel_count', 'pixel_list'],encoding='utf-8')

In [18]:
insert into tbl_eggcount(egg_total_df['왕란'][0], egg_total_df['특란'][0], egg_total_df['대란'][0], egg_total_df['중란'][0], egg_total_df['소란'][0], egg_total_df['경란'][0]) values('{pdRow[1]['create_time']}','{pdRow[1]['house_id']}','{pdRow[1]['module_id']}',{pdRow[1]['weightMedian']},{pdRow[1]['weightMeanStd']},{pdRow[1]['pixelMean']})"

369

In [13]:
egg_df_dict= {}
egg_total_df = pd.DataFrame(columns=["날짜", "왕란", "특란", "대란", "중란", "소란", "경란", "파란","혈란","Sub Total", "Total", "CSV Total", "CSV Good Sum", "CSV Crack Sum"])
# today = datetime.now()
# today = today.strftime("%Y%m%d")

for idx, file_name in enumerate(img_namess):
    date = file_name.split("\\")[-1].split("_")[0]
#     if date != today :
#         continue
    
    egg_dict = {}
    total_count = 0
    with open(file_name, "r") as file :
        line_number= 1

        while True :
            line = file.readline()

            if not line :
                break

            line_strip = line.strip()

            if line_strip == '':
                continue

            else :
                if line_number == 1 :
                    value = line.strip().split(",")
                    for i in range(0, len(value)) :
                        if i == 0 :
                            continue

                        val = value[i].split(":")
                        total_count = val[1]
                    #pass
                else :
                    value = line.strip().split(":")
                    egg_dict[value[0]] = value[1]

            line_number +=1
    
    # DataFrame 생성
    egg_df = pd.DataFrame([egg_dict]).T
    egg_df = egg_df[egg_df.columns.values.tolist()[0]].str.split(',')
    egg_df = egg_df.apply(lambda x: pd.Series(x))
    egg_df = egg_df.rename(columns={0:"왕란", 1:"특란", 2:"대란", 3:"중란", 4:"소란", 5:"경란", 6:"파란", 7:"혈란"})
    egg_df.loc['단위중량'] = egg_df.apply(lambda x : round(int(x.loc['등급중량']) / int(x.loc['등급수량']) , 2) if (int(x.loc['등급중량'])!=0) & (int(x.loc['등급수량'])!=0)else 0 )
    egg_df = egg_df.astype(int)
    egg_df['Sub Total'] = egg_df['왕란'] + egg_df['특란'] + egg_df['대란']  + egg_df['중란'] + egg_df['소란'] + egg_df['경란']
    egg_df['Total'] = total_count
    egg_df = egg_df.loc['등급수량']
    egg_df['날짜'] = date
    
    # CSV 데이터 붙이기
    try : 
        egg_df_csv = pd.read_csv(file_name.replace("_G00_N00.txt",".csv"), engine='python')
        egg_df_csv = egg_df_csv[['Total','Good','Crack']]

        egg_df['CSV Total'] = egg_df_csv['Total'].sum()
        egg_df['CSV Good Sum'] = egg_df_csv['Good'].sum()
        egg_df['CSV Crack Sum'] = egg_df_csv['Crack'].sum()

        egg_df = egg_df[["날짜", "왕란", "특란", "대란", "중란", "소란", "경란","파란","혈란", "Sub Total", "Total", "CSV Total", "CSV Good Sum", "CSV Crack Sum"]]
        #egg_total_df = egg_df.tolist()
        egg_total_df.loc[len(egg_total_df)] = list(egg_df)
    except Exception as e :
        print(e)
        egg_df['CSV Total'] = 0
        egg_df['CSV Good Sum'] = 0
        egg_df['CSV Crack Sum'] = 0
        print(egg_df)
egg_total_df
#egg_total_df[(egg_total_df['소란'] > 0) | (egg_total_df['경란'] > 0)]

[Errno 2] No such file or directory: 'C:\\Users\\USER\\Desktop\\데이터\\알찬\\eggcounter\\20220126\\20220125.csv'
왕란                    476
특란                  12802
대란                    365
중란                     13
소란                      0
경란                      0
파란                      8
혈란                    304
Sub Total           13656
Total               13968
날짜               20220125
CSV Total               0
CSV Good Sum            0
CSV Crack Sum           0
Name: 등급수량, dtype: object


,날짜,왕란,특란,대란,중란,소란,경란,파란,혈란,Sub Total,Total,CSV Total,CSV Good Sum,CSV Crack Sum
0,20220101,41,3791,46,4,0,0,25,72,3882,3979,3977,3871,106
1,20220102,44,15756,60,73,18,0,34,212,15951,16198,16007,15733,274
2,20220103,190,12354,285,2,0,0,16,151,12831,13000,12940,12736,204
3,20220104,201,11473,138,0,0,0,17,95,11812,11926,11633,11501,132
4,20220105,394,16885,73,0,0,0,42,464,17352,17862,17741,17165,576
5,20220106,200,5561,115,0,0,0,10,69,5876,5955,5777,5667,110
6,20220107,112,15303,116,0,0,0,8,179,15531,15719,15663,15421,242
7,20220108,41,3772,29,0,0,0,10,45,3842,3898,3898,3840,58
8,20220109,48,7126,30,0,0,0,7,93,7204,7304,7300,7181,119
9,20220110,437,13892,74,0,0,0,9,165,14403,14581,14559,14349,210


In [7]:
# :1, :21208
# : 0, 0, 0, 0, 0, 0, 0,0
sum([626, 19480, 417, 13, 0, 0, 104, 565])
# : 47613, 1228500, 19984, 536, 0, 0, 9134, 33378

9134 / 104
33378 / 565

21205

87.82692307692308

59.07610619469027

In [10]:
3246 / 41
239958 /3791
2350 / 46
153 / 4
0
0
2364 / 25
4310 / 72


3246, 239958, 2350, 153, 0, 0, 2364, 4310
41, 3791, 46, 4, 0, 0, 25, 72

79.17073170731707

63.296755473489846

51.08695652173913

38.25

0

0

94.56

59.861111111111114

(3246, 239958, 2350, 153, 0, 0, 2364, 4310)

(41, 3791, 46, 4, 0, 0, 25, 72)

In [70]:
sum([41, 3791, 46, 4, 0, 0, 25, 72])

3979

In [71]:
sum([41, 3791, 46, 4,])

3882